In [1]:
import pandas as pd

In [2]:
trainSet = pd.read_csv("/Users/vikaspuri/ArtificialIntelligence/MachineLearningApplications/PlantDiseases/train.csv")
testSet = pd.read_csv("/Users/vikaspuri/ArtificialIntelligence/MachineLearningApplications/PlantDiseases/test.csv")

In [3]:
def append(fileName):
    return fileName + ".jpg"
    

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
trainSet["image_id"]

0          Train_0
1          Train_1
2          Train_2
3          Train_3
4          Train_4
           ...    
1816    Train_1816
1817    Train_1817
1818    Train_1818
1819    Train_1819
1820    Train_1820
Name: image_id, Length: 1821, dtype: object

In [6]:
trainSet["image_id"] = append(trainSet["image_id"])
testSet["image_id"] = append(testSet["image_id"])

In [7]:
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale = 1/255, validation_split = 0.25)

In [8]:
trainSet

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0.jpg,0,0,0,1
1,Train_1.jpg,0,1,0,0
2,Train_2.jpg,1,0,0,0
3,Train_3.jpg,0,0,1,0
4,Train_4.jpg,1,0,0,0
...,...,...,...,...,...
1816,Train_1816.jpg,0,0,0,1
1817,Train_1817.jpg,1,0,0,0
1818,Train_1818.jpg,1,0,0,0
1819,Train_1819.jpg,0,0,1,0


In [9]:
trainData = trainSet[:1366]

In [10]:
train_generator=datagen.flow_from_dataframe(
dataframe=trainData,
directory= '/Users/vikaspuri/ArtificialIntelligence/MachineLearningApplications/PlantDiseases/images',
x_col="image_id",
y_col= ['healthy', 'multiple_diseases', 'rust', 'scab'],
subset="training",
batch_size=32,
seed=50,
shuffle=True,
class_mode="raw",
target_size=(32,32))

Found 1025 validated image filenames.


In [11]:
testData = trainSet[1366:]

In [12]:
valid_generator=datagen.flow_from_dataframe(
dataframe=testData,
directory= '/Users/vikaspuri/ArtificialIntelligence/MachineLearningApplications/PlantDiseases/images',
x_col="image_id",
y_col= ['healthy', 'multiple_diseases', 'rust', 'scab'],
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="raw",
target_size=(32,32))

Found 113 validated image filenames.


In [13]:
test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=testSet,
directory="/Users/vikaspuri/ArtificialIntelligence/MachineLearningApplications/PlantDiseases/images",
x_col="image_id",
y_col=None,
batch_size=32,
seed=42,
shuffle=False,
class_mode=None,
target_size=(32,32))

Found 1821 validated image filenames.


In [14]:
from tensorflow.keras import datasets, layers, models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
Total params: 56,320
Trainable params: 56,320
Non-trainable params: 0
_________________________________________________________________


model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

In [16]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(4))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(trainSet, ['healthy', 'multiple_diseases', 'rust', 'scab'], epochs=10, 
                    validation_data=(test_images, test_labels))